In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import gridspec
rc('text', usetex=False)
plt.rcParams.update({'font.size': 8})
import scipy
from scipy.interpolate import Rbf, interp1d, griddata
from scipy.signal import find_peaks
from scipy.optimize import least_squares
import os
import seaborn as sns
import pandas as pd
import flapjack
#plt.rcParams['figure.figsize'] = 5, 4

### Define the models of the characterization circuits

In [ ]:
def step_receiver(fp, A, alpha_A, K_A, n_A, mu, gamma, Dt):
    # Update protein concs
    a = (A/K_A)**n_A
    nextfp = fp + (alpha_A*a/(1 + a) - gamma*fp - mu*fp) * Dt
    return nextfp

In [ ]:
def step_inverter(p_i, fp, alpha_j, K_i, n_i, mu, gamma, Dt):
    p = (p_i/K_i)**n_i
    nextfp = fp + ( alpha_j / ( 1 + p ) - gamma*fp - mu*fp ) * Dt
    return nextfp

In [ ]:
def step_FFL(p_i, p_j, fp, alpha_k, K_j, n_j, mu, gamma, Dt):
    p = (p_j/K_j)**n_j
    nextfp = fp + ( alpha_k / ( 1 + p ) - gamma*fp - mu*fp ) * Dt
    return nextfp

### The characterization model for one edge
We may consider the repressilator as a graph with each repressor as a node, and each edge an interaction between a repressor and a promoter. In this case we must characterize each edge of the graph, each interaction between repressor and promoter. To solve for the parameters we use a forward model of the data simulating a plate reader.

In [ ]:
def characterize(
    alpha_j=1, alpha_k=1, alpha_A=1,
    n_i=2, n_j=2,
    K_i=1, K_j=1, K_A=1,
    Dt=0.05,
    sim_steps=10,
    A=[0],
    muval=[1]*100, odval=[1]*100
):
    p1_list,p2_list,p3_list,A_list,t_list = [],[],[],[],[]
    p1 = np.zeros_like(A)
    p2 = np.zeros_like(A)
    p3 = np.zeros_like(A)
    for t in range(100):
        od = odval[t]
        mu = muval[t]
        for tt in range(sim_steps):
            nextp1 = step_receiver(p1, A, alpha_A, K_A, mu, Dt/sim_steps)
            nextp2 = step_inverter(p1, p2, alpha_j, K_i, n_i, mu, Dt/sim_steps)
            nextp3 = step_FFL(p1, p2, p3, alpha_k, K_j, n_j, mu, Dt/sim_steps)
            p1,p2,p3 = nextp1,nextp2,nextp3

        p1_list.append(p1*od)
        p2_list.append(p2*od)
        p3_list.append(p3*od)
        A_list.append(A)
        t_list.append([t * Dt]*len(A))

    ap1 = np.array(p1_list).transpose()
    ap2 = np.array(p2_list).transpose()
    ap3 = np.array(p3_list).transpose()
    AA = np.array(A_list).transpose()
    tt = np.array(t_list).transpose()
    t = np.arange(100) * Dt
    return ap1,ap2,ap3,AA,tt

In [ ]:
def characterize_receiver(
    alpha_A=1,
    K_A=1,
    n_A=2,
    Dt=0.05,
    sim_steps=10,
    A=[0],
    muval=[1]*100, odval=[1]*100,
    gamma=0
):
    p1_list,A_list,t_list = [],[],[]
    p1 = np.zeros_like(A)
    for t in range(100):
        od = odval[t]
        mu = muval[t]
        for tt in range(sim_steps):
            nextp1 = step_receiver(p1, A, alpha_A, K_A, n_A, mu, gamma, Dt/sim_steps)
            p1 = nextp1

        p1_list.append(p1*od)
        A_list.append(A)
        t_list.append([t * Dt]*len(A))

    ap1 = np.array(p1_list).transpose()
    AA = np.array(A_list).transpose()
    tt = np.array(t_list).transpose()
    t = np.arange(100) * Dt
    return ap1,AA,tt

In [ ]:
def characterize_inverter(
    alpha_j=1,
    n_i=2,
    K_i=1,
    alpha_A=1e2,
    K_A=1,
    n_A=2,
    Dt=0.05,
    sim_steps=10,
    A=[0],
    muval=[1]*100, odval=[1]*100,
    gamma=0
):
    p2_list,A_list,t_list = [],[],[]
    p1 = np.zeros_like(A)
    p2 = np.zeros_like(A)
    for t in range(100):
        od = odval[t]
        mu = muval[t]
        for tt in range(sim_steps):
            nextp1 = step_receiver(p1, A, alpha_A, K_A, n_A, mu, gamma, Dt/sim_steps)
            nextp2 = step_inverter(p1, p2, alpha_j, K_i, n_i, mu, 0, Dt/sim_steps)
            p1,p2 = nextp1,nextp2

        p2_list.append(p2*od)
        A_list.append(A)
        t_list.append([t * Dt]*len(A))

    ap2 = np.array(p2_list).transpose()
    AA = np.array(A_list).transpose()
    tt = np.array(t_list).transpose()
    t = np.arange(100) * Dt
    return ap2,AA,tt

### Plot the three characterization circuit models

In [ ]:
alpha_1 = 1
alpha_2 = 1
alpha_3 = 1
alpha_A = 1
n_1 = 2
n_2 = 2
n_3 = 2
K_1 = 1
K_2 = 1
K_3 = 1
K_A = 1
Dt = 0.24
A = np.logspace(-3, 3, 3, endpoint=True)

ap1,ap2,ap3,AA,tt = characterize(
    alpha_j=alpha_2, alpha_k=alpha_3, alpha_A=alpha_A,
    n_i=n_1, n_j=n_2,
    K_i=K_1, K_j=K_2, K_A=K_A,
    Dt=Dt,
    A=A
)

width = 40 / 25.4
height = 120 / 25.4
fig,axs = plt.subplots(3,1,figsize=(width,height), sharex=True)

df = pd.DataFrame()
df['p1'] = ap1.ravel()
df['p2'] = ap2.ravel()
df['p3'] = ap3.ravel()
df['Time'] = tt.ravel()
df['A'] = AA.ravel()
df['logA'] = np.log10(AA.ravel())

df.sort_values('Time').groupby('A').plot(x='Time', y='p1', ax=axs[0], style='-')
axs[0].legend(['A = %g'%a for a in A], loc=(0,1.1))
df.sort_values('Time').groupby('A').plot(x='Time', y='p2', ax=axs[1], style='-')
axs[1].legend().remove()
df.sort_values('Time').groupby('A').plot(x='Time', y='p3', ax=axs[2], style='-')
axs[2].legend().remove()
for i in range(3):
    axs[i].set_xlim([0,4])
    #axs[i].set_xticks([])
    #axs[i].set_yticks([])
    axs[i].set_ylabel('YFP fluo. (AU)')

plt.subplots_adjust(top=0.8)
plt.show()
plt.savefig('repressilator_char.png', dpi=300)

### Characterizing all three edges
The complete characterization model for the repressilator consists of three edge models as described above.

In [ ]:
def repressilator(
    alpha_1=1, alpha_2=1, alpha_3=1, alpha_A=1,
    n_1=2, n_2=2, n_3=2,
    K_1=1, K_2=1, K_3=1, K_A=1,
    A=[0], muval=[1]*100, odval=[1]*100, Dt=0.24
):
    p1_1,p2_1,p3_1,AA,tt = characterize(
        alpha_j=alpha_2, alpha_k=alpha_3, alpha_A=alpha_A,
        n_i=n_1, n_j=n_2,
        K_i=K_1, K_j=K_2, K_A=K_A,
        Dt=Dt, A=A, muval=muval, odval=odval
    )
    p1_2,p2_2,p3_2,AA,tt = characterize(
        alpha_j=alpha_3, alpha_k=alpha_1, alpha_A=alpha_A,
        n_i=n_2, n_j=n_3,
        K_i=K_2, K_j=K_3, K_A=K_A,
        Dt=Dt, A=A, muval=muval, odval=odval
    )
    p1_3,p2_3,p3_3,AA,tt = characterize(
        alpha_j=alpha_1, alpha_k=alpha_2, alpha_A=alpha_A,
        n_i=n_3, n_j=n_1,
        K_i=K_3, K_j=K_1, K_A=K_A,
        Dt=Dt, A=A, muval=muval, odval=odval
    )
    return( np.concatenate((p1_1,p2_1,p3_1,p1_2,p2_2,p3_2,p1_3,p2_3,p3_3)).ravel() )

### Define the residuals for model fitting

In [ ]:
def residuals(data, A, muval, odval, Dt): 
    def func(x): 
        alpha_1 = x[0]
        alpha_2 = x[1]
        alpha_3 = x[2]
        alpha_A = x[3]
        n_1,n_2,n_3 = 2,2,2 #x[4:7]
        K_1,K_2,K_3,K_A = x[4:8]
        return data - repressilator(
                        alpha_1=alpha_1, alpha_2=alpha_2, alpha_3=alpha_3, alpha_A=alpha_A,
                        n_1=n_1, n_2=n_2, n_3=n_3,
                        K_1=K_1, K_2=K_2, K_3=K_3, K_A=K_A,
                        A=A, muval=muval, odval=odval, Dt=Dt
                    )
    return func

### Create some test data and fit the model

In [ ]:
A = 10**(np.arange(24)/2-6)
t = np.linspace(0,24,200)
odval = flapjack.gompertz(t, 0.01, 1, 1, 4)
muval = flapjack.gompertz_growth_rate(t, 0.01, 1, 1, 4)
fake_data = repressilator(A=A, 
                     alpha_1=1, alpha_2=1, alpha_3=1,
                     K_1=1, K_2=1, K_3=1, 
                     alpha_A=1e2, 
                     muval=muval, odval=odval)
#data = data + np.random.normal(0,0.01,size=data.shape)

In [ ]:
plt.plot(fake_data[:100])

In [ ]:
lower_bounds = [0]*8
upper_bounds = [1e4]*4 + [1000]*4
bounds = [lower_bounds, upper_bounds]
res_1 = least_squares(residuals(fake_data, A, muval, odval, Dt), [2]*8, bounds=bounds)

In [ ]:
x = res_1.x
alpha_1 = x[0]
alpha_2 = x[1]
alpha_3 = x[2]
alpha_A = x[3]
#n_1,n_2,n_3 = x[4:7]
K_1,K_2,K_3,K_A = x[4:8]
print(res_1.x)

### Characterize via flapjack

In [ ]:
def residuals(data, A, muval, odval, Dt): 
    def func(x): 
        alpha_j = x[0]
        alpha_k = x[1]
        alpha_A = x[2]
        #n_i,n_j = x[3:5]
        n_i,n_j = 2,2
        K_i,K_j,K_A = x[3:6]
        p1,p2,p3,AA,tt = characterize(
                    alpha_j=alpha_j, alpha_k=alpha_k, alpha_A=alpha_A,
                    n_i=n_i, n_j=n_j,
                    K_i=K_i, K_j=K_j, K_A=K_A,
                    Dt=Dt,
                    A=A,
                    muval=muval, odval=odval
                )
        model = np.concatenate((p1,p2,p3)).ravel()
        return data - model
    return func

In [ ]:
def residuals_receiver(data, A, muval, odval, Dt): 
    def func(x): 
        alpha_A = x[0]
        K_A = x[1]
        n_A = x[2]
        
        p,AA,tt = characterize_receiver(
                    alpha_A=alpha_A,
                    K_A=K_A,
                    n_A=n_A,
                    Dt=Dt,
                    A=A,
                    muval=muval, odval=odval
                )
        model = p.ravel()
        return data - model
    return func

In [ ]:
def residuals_inverter(data, alpha_A, K_A, n_A, A, muval, odval, Dt): 
    def func(x): 
        alpha_j = x[0]
        n_i = x[1]
        K_i = x[2]
        gamma = x[3]

        p,AA,tt = characterize_inverter(
                    alpha_j=alpha_j,
                    n_i=n_i,
                    K_i=K_i,
                    alpha_A=alpha_A,
                    K_A=K_A,
                    n_A=n_A,
                    Dt=Dt,
                    A=A,
                    muval=muval, odval=odval,
                    gamma=gamma
                )
        model = p.ravel()
        return data - model
    return func

In [ ]:
import flapjack
fj = flapjack.Flapjack(url_base='localhost:8000')
fj.log_in(username='tim', password='chicken')

In [ ]:
receiver_study = fj.get('study', name=['Receiver simulation alpha scan deg'])
receiver_df = fj.measurements(study=receiver_study.id)
receiver_df = receiver_df.sort_values(['Sample', 'Concentration1', 'Time'])
receiver = receiver_df[receiver_df.Signal=='SFP0'].groupby(['Concentration1', 'Time']).mean().Measurement.values

In [ ]:
fj.get('chemical')

In [ ]:
odval = receiver_df[receiver_df.Signal=='SOD'].groupby(['Time']).mean().Measurement.values
od_signal = fj.get('signal', name='SOD')
mu = fj.analysis(type='Expression Rate (direct)',
                eps_L=1e-3,
                degr=0,
                 biomass_signal=od_signal.id[0],
                 study=receiver_study.id,
                 signal=od_signal.id
                )
muval = mu.groupby('Time').mean().Rate.values
mutimes = mu.groupby('Time').mean().index.values
times = np.linspace(0, 48, 200)
from scipy.interpolate import interp1d
imu = interp1d(mutimes, muval, 1, bounds_error=False, fill_value='extrapolate')
muval = imu(times)

In [ ]:
def get_inverter(
    alpha_j = 1e2,
    K_i = 1,
    n_i = 2,
    gamma=0
):
    study = fj.get('study', name='TU characterization alpha/gamma scan 2')
    assay_name = f'Inverter, alpha_j={alpha_j}, K_i={K_i}, n_i={n_i}, gamma={gamma}'
    inverter_assay = fj.get(
        'assay', 
        name=[assay_name],
        study=study.id
    )
    inverter = fj.measurements(assay=[inverter_assay.id[0]])
    inverter = inverter.sort_values(['Sample', 'Concentration1', 'Time'])
    inverter = inverter[inverter.Signal=='SFP0'].groupby(['Concentration1', 'Time']).mean().Measurement.values
    return inverter

In [ ]:
def get_dbl_inverter(
    alpha_j = 1e2,
    K_j = 1,
    n_j = 2,
):
    study = fj.get('study', name='TU characterization alpha scan deg 2')
    dbl_inverter_assay = fj.get(
        'assay', 
        name=[f'Double inverter, alpha_j={alpha_j}, K_j={K_j}, n_j={n_j}'],
        study=study.id
    )
    dbl_inverter = fj.measurements(assay=[dbl_inverter_assay.id[0]])
    dbl_inverter = dbl_inverter.sort_values(['Sample', 'Concentration1', 'Time'])
    dbl_inverter = dbl_inverter[dbl_inverter.Signal=='SFP0'].groupby(['Concentration1', 'Time']).mean().Measurement.values
    return dbl_inverter

In [ ]:
params = {}
alphas = np.logspace(2, 5, 4, endpoint=True)
# Loop over alpha for 1st TU
for alpha in alphas:
    # Get the data for fitting
    inverter = get_inverter(alpha_j=alpha, K_i=1, n_i=2)
    dbl_inverter = get_dbl_inverter(alpha_j=alpha, K_j=1, n_j=2)
    data = np.concatenate([
                receiver, inverter, dbl_inverter,
            ])
    data = data.ravel()

    # Concentration range
    A = np.logspace(-6, 6, 24, endpoint=True)
    
    # Bounds for fitting
    lower_bounds = [0]*6
    upper_bounds = [1e8]*3 + [100]*3
    bounds = [lower_bounds, upper_bounds]
    '''
        alpha_j = x[0]
        alpha_k = x[1]
        alpha_A = x[2]
        n_i,n_j = x[3:5]
        K_i,K_j,K_A = x[5:7]
    '''
    res = least_squares(residuals(data, A, muval, odval, 0.24), [1]*6, bounds=bounds)
    print(alpha, res.x)
    params[alpha] = res.x

### Characterize receiver only

In [ ]:
# Concentration range
A = np.logspace(-6, 6, 24, endpoint=True)

# Bounds for fitting
lower_bounds = [0]*3
upper_bounds = [1e8, 100, 4]
bounds = [lower_bounds, upper_bounds]
'''
    alpha_A = x[0]
    K_A = x[1]
'''
data = receiver.ravel()
res = least_squares(residuals_receiver(data, A, muval, odval, 0.24), [1]*3, bounds=bounds)
print(res.x)
alpha_A = res.x[0]
K_A = res.x[1]
n_A = res.x[2]

### Characterize inverter

In [ ]:
# Concentration range
A = np.logspace(-6, 6, 24, endpoint=True)

# Bounds for fitting
lower_bounds = [0]*4
upper_bounds = [1e8, 4, 100, 6]
bounds = [lower_bounds, upper_bounds]
'''
    alpha_j = x[0]
    n_i = x[1]
    K_i = x[2]
    gamma = x[3]
'''
alphas = np.logspace(2, 5, 4, endpoint=True)
gammas = np.linspace(0.5, 2, 3, endpoint=True)
# Loop over alpha for 1st TU
params = {}
for gamma in gammas:
    for alpha in alphas:
        inverter = get_inverter(alpha_j=alpha, K_i=1, n_i=2, gamma=gamma)
        data = inverter.ravel()
        res = least_squares(residuals_inverter(data, alpha_A, K_A, n_A, A, muval, odval, 0.24), [1]*4, bounds=bounds)
        params[(alpha, gamma)] = res.x
        print(res.x)

In [ ]:
print(params)

### Predict the repressilator

In [ ]:
def step(
    p, 
    alpha1, alpha2, alpha3, 
    gamma1, gamma2, gamma3, 
    K1, K2, K3, 
    n1, n2, n3, 
    growth_rate, dt
):
    '''
    Update protein levels p according to signal
    '''
    p1,p2,p3 = p
    dp1dt = alpha1 / (1 + (p3/K3)**n1) - gamma1 * p1 - growth_rate * p1
    dp2dt = alpha2 / (1 + (p1/K1)**n2) - gamma2 * p2 - growth_rate * p2
    dp3dt = alpha3 / (1 + (p2/K2)**n3) - gamma3 * p3 - growth_rate * p3
    p[0] += dp1dt * dt
    p[1] += dp2dt * dt
    p[2] += dp3dt * dt
    return p

### Choose some random combinations of TUs

In [ ]:
import random
# Make some random repressilators
sample_params = {}
for i in range(1728):
    key1, sample_params1 = random.choice(list(params.items()))
    key2, sample_params2 = random.choice(list(params.items()))
    key3, sample_params3 = random.choice(list(params.items()))
    key = key1 + key2 + key3
    alpha1 = sample_params1[0]
    alpha2 = sample_params2[0]
    alpha3 = sample_params3[0]
    n1 = sample_params3[1]
    n2 = sample_params1[1]
    n3 = sample_params2[1]
    K1 = sample_params3[2]
    K2 = sample_params1[2]
    K3 = sample_params2[2]
    gamma1 = sample_params1[3]
    gamma2 = sample_params2[3]
    gamma3 = sample_params3[3]
    sample_params[key] = (
        alpha1, alpha2, alpha3,
        n1, n2, n3,
        K1, K2, K3,
        gamma1, gamma2, gamma3
    )

### Simulate the repressilator with fitted parameters

In [ ]:
'''
    alpha_j = x[0]
    n_i = x[1]
    K_i = x[2]
    gamma = x[3]
'''

dt = 24/100

T_models = {}
p_models = {}
for key,p in sample_params.items():
    alpha1,alpha2,alpha3,n1,n2,n3,K1,K2,K3,gamma1,gamma2,gamma3 = p
    pp = [0,5,0] #np.array([0, 5, 0])
    p1_list = []
    p2_list = []
    p3_list = []
    for t in range(int(24/dt)):
        od = flapjack.gompertz(t*dt, 0.01, 1, 1, 4)
        p1_list.append(pp[0]*od)
        p2_list.append(pp[1]*od)
        p3_list.append(pp[2]*od)
        for tt in range(10):
            growth_rate = flapjack.gompertz_growth_rate((t + tt/10)*dt, 0.01, 1, 1, 4)
            pp = step(pp, 
                     alpha1=alpha1, 
                     alpha2=alpha2, 
                     alpha3=alpha3, 
                     gamma1=gamma1, 
                     gamma2=gamma2, 
                     gamma3=gamma3,
                     K1=K1, 
                     K2=K2, 
                     K3=K3, 
                     n1=n1, 
                     n2=n2,
                     n3=n3,
                     growth_rate=growth_rate, dt=dt/10)
    p1 = np.array(p1_list)
    p2 = np.array(p2_list)
    p3 = np.array(p3_list)
    p_model = np.stack([p1,p2,p3]).transpose()
    p_models[key] = p_model

    pks_model = find_peaks(p_model[:,0], height=0.1*p_model[:,0].max())[0]
    if len(pks_model)>2:
        T_model = dt * np.diff(pks_model).mean()
    else:
        T_model = np.nan
    T_models[key] = T_model

### Test the repressilator designs

In [ ]:

def step_repressilator(alpha1, alpha2, alpha3): 
    def func(p, signal, growth_rate, dt):
        nextp = step(p, alpha1, alpha2, alpha3, 1, 1, 1, 1, 2, 2, 2, growth_rate, dt)
        return nextp
    return func

alphas = [1e3]
study_name = 'Simulated repressilators 10'
for alpha1_true in alphas:
    for alpha2_true in alphas:
        for alpha3_true in alphas:
            assay_name = f'alpha1={alpha1_true}, alpha2={alpha2_true}, alpha3={alpha3_true}'
            sim = flapjack.Simulator(
                    study_name=study_name,
                    assay_name=assay_name,
                    study_description='Simulation of repressilators',
                    assay_description='Replicate 2',
                    dna_name='pRPR',
                    init_proteins=[0,5,0],
                    concentrations=[0],
                    n_signals=3,
                    fluo_noise=0.01,
                    od_noise=0.01
                )
            sim.create_meta_objects(fj)
            sim.create_data(fj, step_repressilator(alpha1_true, alpha2_true, alpha3_true), 1, 200, 24/100, 10)

In [ ]:
signal = fj.get('signal', name=f'SFP{3}')
signal

In [ ]:
assay_name = f'alpha1={alphas[0]}, alpha2={alphas[0]}, alpha3={alphas[0]}'
assay = fj.get('assay', name='assay_name')
assay

In [ ]:
dt = 24/100
K1 = 1
K2 = 1
K3 = 1
n1 = 2
n2 = 2
n3 = 2
T_trues = {}
p_trues = {}
for key,p in sample_params.items():
    pp = [0,5,0] #np.array([0, 5, 0])
    p1_list = []
    p2_list = []
    p3_list = []
    alpha1,gamma1,alpha2,gamma2,alpha3,gamma3 = key
    for t in range(int(24/dt)):
        od = flapjack.gompertz(t*dt, 0.01, 1, 1, 4)
        p1_list.append(pp[0]*od)
        p2_list.append(pp[1]*od)
        p3_list.append(pp[2]*od)
        for tt in range(10):
            growth_rate = flapjack.gompertz_growth_rate((t + tt/10)*dt, 0.01, 1, 1, 4)
            pp = step(pp, 
                     alpha1=alpha1, 
                     alpha2=alpha2, 
                     alpha3=alpha3, 
                     gamma1=gamma1, 
                     gamma2=gamma2, 
                     gamma3=gamma3, 
                     K1=K1, 
                     K2=K2, 
                     K3=K3, 
                     n1=n1, 
                     n2=n2,
                     n3=n3,
                     growth_rate=growth_rate, dt=dt/10)
    p1 = np.array(p1_list)
    p2 = np.array(p2_list)
    p3 = np.array(p3_list)
    p_true = np.stack([p1,p2,p3]).transpose()
    p_trues[key] = p_true
    pks_true = find_peaks(p_true[:,0], height=0.1*p_true[:,0].max())[0]
    if len(pks_true)>2:
        T_true = dt * np.diff(pks_true).mean()
    else:
        T_true = np.nan
    T_trues[key] = T_true

In [ ]:
#error = np.array(p_trues) - np.array(p_models)

### Calculate the error in our estimate

Error in oscillation periods:

In [ ]:
error = np.array(list(T_trues.values())) - np.array(list(T_models.values()))

rmse = np.sqrt(np.nanmean(error**2)) 
print(f'RMSE = {rmse}')

nrmse = np.sqrt(np.nanmean(error**2)) / np.nanmean(list(T_trues.values()))
print(f'NRMSE = {nrmse}')

mae = np.mean(np.abs(error[~np.isnan(error)]))
print(f'Mean absolute error (MAE) = {mae}')

msignede = np.nanmean(error)
print(f'Mean signed error = {msignede}')

Error in timeseries:

In [ ]:
error = np.array(list(p_trues.values()) - np.array(list(p_models.values())))

rmse = np.sqrt(np.nanmean(error**2)) 
print(f'RMSE = {rmse}')

nrmse = np.sqrt(np.nanmean(error**2)) / np.nanmean(list(p_trues.values()))
print(f'NRMSE = {nrmse}')

mae = np.mean(np.abs(error[~np.isnan(error)]))
print(f'Mean absolute error (MAE) = {mae}')

msignede = np.nanmean(error)
print(f'Mean signed error = {msignede}')

### Plot an example repressilator

In [ ]:
dt = 24/100
key = list(sample_params.keys())[25]
fig,ax = plt.subplots(1,1, figsize=(7,4))
ax.plot(np.arange(100)*dt, p_models[key][:,0], 'r-')
ax.plot(np.arange(100)*dt, p_models[key][:,1], 'g-')
ax.plot(np.arange(100)*dt, p_models[key][:,2], 'b-')
ax.plot(np.arange(100)*dt, p_trues[key][:,0], 'r--+')
ax.plot(np.arange(100)*dt, p_trues[key][:,1], 'g--+')
ax.plot(np.arange(100)*dt, p_trues[key][:,2], 'b--+')
T = T_models[key]
T_data = T_trues[key]
ax.set_title('$T_{{model}}$ = %0.2g, $T_{{data}}$ = %0.2g'%(T, T_data) if not np.isnan(T) else 'Non-oscillatory')
print(key)

### Plot some repressilators

In [ ]:
fig,axs = plt.subplots(4, 4, figsize=(7,4))
axs = axs.ravel()
keys = list(sample_params.keys())
for idx,i in enumerate(np.random.randint(len(keys), size=(16,))):
    k = keys[i]
    ax = axs[idx]
    ax.plot(np.arange(100)*dt, p_models[k][:,0], 'r-', linewidth=1)
    ax.plot(np.arange(100)*dt, p_models[k][:,1], 'g-', linewidth=1)
    ax.plot(np.arange(100)*dt, p_models[k][:,2], 'b-', linewidth=1)
    ax.plot(np.arange(100)*dt, p_trues[k][:,0], 'r--', linewidth=1)
    ax.plot(np.arange(100)*dt, p_trues[k][:,1], 'g--', linewidth=1)
    ax.plot(np.arange(100)*dt, p_trues[k][:,2], 'b--', linewidth=1)
    ax.set_xticks([])
    ax.set_yticks([])
    #ax.set_title(k)
plt.tight_layout()
plt.savefig('repressilator_dynamics.png', dpi=300)

In [ ]:
T_true_vals = np.array(list(T_trues.values()))
T_model_vals = np.array(list(T_models.values()))
plt.plot(T_true_vals, T_model_vals, '+', markersize=14)
plt.plot([2,6], [2,6], 'k--')
plt.xlabel('Period (data)')
plt.ylabel('Period (fitted model)')

plt.figure()
idx = np.where(~np.isnan(T_true_vals*T_model_vals)) 
hist,xedges,yedges = np.histogram2d(T_true_vals[idx], T_model_vals[idx], bins=16)
#plt.hist2d(T_true_vals[idx], T_model_vals[idx], (16, 16), cmap=plt.cm.jet);
plt.imshow(hist)
plt.xticks(ticks=np.arange(16), labels=np.round(xedges, 1))
plt.yticks(yedges)
plt.colorbar()

In [ ]:
T = list(T_trues.values())
true_oscillators = np.where(~np.isnan(T))[0]

In [ ]:
T = list(T_models.values())
model_oscillators = np.where(~np.isnan(T))[0]

In [ ]:
len(true_oscillators)

In [ ]:
true_oscillators == model_oscillators

In [ ]:
false_positives = set(model_oscillators).difference(true_oscillators)
len(false_positives)

In [ ]:
false_negatives = set(true_oscillators).difference(model_oscillators)
len(false_negatives)